# MALIGNANT COMMENTS CLASSIFIER

In [1]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load datasets
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
train

In [ ]:
test

In [ ]:
print('Train_data:',train.shape)
print('Test_data:',test.shape)

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

# Visualizations

Count plot:

In [ ]:
col=['malignant','highly_malignant','rude','threat','abuse','loathe']

for i in col:
    print(i)
    print("\n")
    print(train[i].value_counts())
    sns.countplot(train[i])
    plt.show()

From the above observation we can see that, 15294 comments are malignant and 144277 comments are not malignant.
1595 comments are highly_malignant and 157976 comments are not highly_malignant.
8449 comments are rude and 151122 are not rude.
478 comments are threat and 159093 comments are not threat.
7877 comments are abuse and 151694 comments are not abuse.
1405 comments are loathe and 158166 comments are not loathe.

# Correlation:

In [ ]:
train.corr()

correlation using heatmap:

In [ ]:
plt.figure(figsize=(15,7))
sns.heatmap(train.corr(),annot=True,linewidth=0.5,linecolor="black",fmt='.2f')

# Describing dataset:

In [ ]:
train.describe()

Describe of dataset using heatmap:

In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(round(train.describe()[1:].transpose(),2),linewidth=2,annot=True,fmt="f")
plt.xticks(fontsize=18)
plt.yticks(fontsize=12)
plt.title("Variable Summary")
plt.show()

In [ ]:
print(train.info())
print("\n")
print(test.info())

# Skewness:

In [ ]:
train.skew()

All the columns of the dataset has skewness.

# Normal Distribution Curve:

In [ ]:
sns.distplot(train['malignant'])

The data of the column is not normalized. The building blocks is out of the normalized curve.

In [ ]:
sns.distplot(train['highly_malignant'])

The data of the column is not normalized. The building blocks is out of the normalized curve.

In [ ]:
sns.distplot(train['rude'])

The data of the column is not normalized. The building blocks is out of the normalized curve.

In [ ]:
sns.distplot(train['threat'])

The data of the column is not normalized. The building blocks is out of the normalized curve.

In [ ]:
sns.distplot(train['abuse'])

The data of the column is not normalized. The building blocks is out of the normalized curve.

In [ ]:
sns.distplot(train['loathe'])

The data of the column is not normalized. The building blocks is out of the normalized curve.

In [ ]:
#importing necessary libraries

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
train['length']=train['comment_text'].str.len()
train

In [ ]:
#convert all messages to lower case
train['comment_text'] = train['comment_text'].str.lower()

#Replace email address with 'email'
train['comment_text'] = train['comment_text'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')

#Replace URLs with 'webaddress'
train['comment_text'] = train['comment_text'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'webaddress')

#Replace money sumbols with 'moneysymb'
train['comment_text'] = train['comment_text'].str.replace(r'£|\$', 'dollars')

#Replace 10 digit phone numbers with 'phonenumber'
train['comment_text'] = train['comment_text'].str.replace(r'\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumber')

#Replace numbers with 'num'
train['comment_text'] = train['comment_text'].str.replace(r'\d+(\.\d+)?', 'num')

train['comment_text'] = train['comment_text'].apply(lambda x: ''.join(term for term in x.split() if term not in string.punctuation))

stop_words = set(stopwords.words('english') + ['u','ur','4','2','im','dont','doin','ure'])
train['comment_text'] = train['comment_text'].apply(lambda x: ''.join(term for term in x.split() if term not in stop_words))

lem=WordNetLemmatizer()
train['comment_text'] = train['comment_text'].apply(lambda x: ''.join(lem.lemmatize(t) for t in x.split()))


In [ ]:
train['clean_length']=train.comment_text.str.len()
train

In [ ]:
#total length removal
print('Original Length: ', train.length.sum())
print('Clean Length: ', train.clean_length.sum())

In [ ]:
#!pip install wordcloud

In [ ]:
#offensive loud words
from wordcloud import WordCloud
hams = train['comment_text'][train['malignant']==1]
spam_cloud = WordCloud(width=600, height=400, background_color='white',max_words=50).generate(''.join(hams))
plt.figure(figsize=(10,8),facecolor='r')
plt.imshow(spam_cloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
target_column=['malignant','highly_malignant','rude','threat','abuse','loathe']

target=train[target_column]

train['bad']=train[target_column].sum(axis=1)
print(train['bad'].value_counts())
train['bad']=train['bad']>0
train['bad']=train['bad'].astype(int)
print(train['bad'].value_counts())

In [ ]:
sns.countplot(x='bad', data=train)
print(train['bad'].value_counts())
plt.show()

In [ ]:
train.head()

In [ ]:
#converting text into vectors using TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(max_features=10000, stop_words='english')
feature= tfidf.fit_transform(train['comment_text'])

In [ ]:
print(train.shape)
print(test.shape)

# Dividing Dataframe into feature and target

In [ ]:
x=feature
y=train['bad']

In [ ]:
y.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE
sm=SMOTE()
xtrain,ytrain=sm.fit_resample(x,y)

In [ ]:
ytrain.value_counts()

# Model Building

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

# Logistic Regression:

In [ ]:
lg=LogisticRegression()

train_x,test_x,train_y,test_y=train_test_split(xtrain,ytrain,test_size=0.20,random_state=45)
lg.fit(train_x,train_y)
pred_train=lg.predict(train_x)
pred_test=lg.predict(test_x)

print("Training Accuracy:",accuracy_score(train_y,pred_train)*100)
print("Testing Accuracy:",accuracy_score(test_y,pred_test)*100)

Cross Validation for logistic regression:

In [ ]:
from sklearn.model_selection import cross_val_score

lg.fit(train_x,train_y)
lg.score(train_x,train_y)
pred_lg = lg.predict(test_x)

lss = accuracy_score(test_y,pred_lg)
for j in range(2,10):
    lsscore = cross_val_score(lg,xtrain,ytrain,cv=j)
    ls_cv = lsscore.mean() 
    print("At cv:-",j)
    print("Cross validation score is:-",ls_cv*100 )
    print("Accuracy score is :-",lss*100)
    print("\n")

In [ ]:
print("At cv:",9)
print("Cross validation score is:",74.33099030511153)
print("Accuracy score is :",74.33858281448926)

AUC-ROC Curve:

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr,tpr,thresholds=roc_curve(pred_lg,test_y)
roc_auc=auc(fpr,tpr)
plt.figure()
plt.plot(fpr,tpr,color='darkorange',lw=10,label='ROC Curve(area=%0.2f)'%roc_auc)
plt.plot([0,1],[0,1],color='navy',lw=10,linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LOGISTIC REGRESSION')
plt.legend(loc="lower right")
plt.show()

# Approaching more classifiers:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# 1.Decision Tree Classifier:

In [ ]:
parameters = {'criterion':['gini', 'entropy'],'splitter':['best','random']}
dtc = DecisionTreeClassifier()
clf = GridSearchCV(dtc,parameters)
clf.fit(train_x,train_y)

print(clf.best_params_)

In [ ]:
dtc = DecisionTreeClassifier(criterion='gini', splitter='best')
dtc.fit(train_x,train_y)
dtc.score(train_x,train_y)
pred_dtc = dtc.predict(test_x)

print("Accuracy Score:",accuracy_score(test_y,pred_dtc)*100)
print("Classification report:",classification_report(test_y,pred_dtc)*100)
print("Confusion Matrix:",confusion_matrix(test_y,pred_dtc)*100)

dtc_score = cross_val_score(dtc,xtrain,ytrain,cv=9)
dtc_cc = dtc_score.mean() 
print('Cross Val Score:',dtc_cc*100)

AUC-ROC Curve:

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr,tpr,thresholds=roc_curve(pred_dtc,test_y)
roc_auc=auc(fpr,tpr)
plt.figure()
plt.plot(fpr,tpr,color='darkorange',lw=10,label='ROC Curve(area=%0.2f)'%roc_auc)
plt.plot([0,1],[0,1],color='navy',lw=10,linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree Classifier')
plt.legend(loc="lower right")
plt.show()

# 2.KNeighbors Classifier:

In [ ]:
parameters = {'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
knc = KNeighborsClassifier()
clf = GridSearchCV(knc,parameters)
clf.fit(train_x,train_y)

print(clf.best_params_)

In [ ]:
knc = KNeighborsClassifier(algorithm='auto', n_neighbors=5)
knc.fit(train_x,train_y)
knc.score(train_x,train_y)
pred_knc = knc.predict(test_x)

print("Accuracy Score:",accuracy_score(test_y,pred_knc)*100)
print("Classification report:",classification_report(test_y,pred_knc)*100)
print("Confusion Matrix:",confusion_matrix(test_y,pred_knc)*100)

knc_score = cross_val_score(knc,xtrain,ytrain,cv=9)
knc_cc = knc_score.mean() 
print('Cross Val Score:',knc_cc*100)

AUC-ROC Curve:

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr,tpr,thresholds=roc_curve(pred_knc,test_y)
roc_auc=auc(fpr,tpr)
plt.figure()
plt.plot(fpr,tpr,color='darkorange',lw=10,label='ROC Curve(area=%0.2f)'%roc_auc)
plt.plot([0,1],[0,1],color='navy',lw=10,linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('KNeighbors Classifier')
plt.legend(loc="lower right")
plt.show()

# ENSEMBLE METHODS:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# 1.Random Forest Classifier:

In [ ]:
parameters = {'criterion':['gini', 'entropy'],'n_estimators':[100]}
rfc = RandomForestClassifier()
clf = GridSearchCV(rfc,parameters)
clf.fit(train_x,train_y)

print(clf.best_params_)

In [ ]:
rfc = RandomForestClassifier(criterion='entropy', n_estimators=100)
rfc.fit(train_x,train_y)
rfc.score(train_x,train_y)
pred_rfc = rfc.predict(test_x)

print("Accuracy Score:",accuracy_score(test_y,pred_rfc)*100)
print("Classification report:",classification_report(test_y,pred_rfc)*100)
print("Confusion Matrix:",confusion_matrix(test_y,pred_rfc)*100)

rfc_score = cross_val_score(rfc,xtrain,ytrain,cv=9)
rfc_cc = rfc_score.mean() 
print('Cross Val Score:',rfc_cc*100)

AUC-ROC Curve

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr,tpr,thresholds=roc_curve(pred_rfc,test_y)
roc_auc=auc(fpr,tpr)
plt.figure()
plt.plot(fpr,tpr,color='darkorange',lw=10,label='ROC Curve(area=%0.2f)'%roc_auc)
plt.plot([0,1],[0,1],color='navy',lw=10,linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Random Forest Classifier')
plt.legend(loc="lower right")
plt.show()

# 2.Ada Boost Classifier:

In [ ]:
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1.0, algorithm='SAMME')
abc.fit(train_x,train_y)
abc.score(train_x,train_y)
pred_abc = abc.predict(test_x)

print("Accuracy Score:",accuracy_score(test_y,pred_abc)*100)
print("Classification report:",classification_report(test_y,pred_abc)*100)
print("Confusion Matrix:",confusion_matrix(test_y,pred_abc)*100)

abc_score = cross_val_score(abc,xtrain,ytrain,cv=9)
abc_cc = abc_score.mean() 
print('Cross Val Score:',abc_cc*100)

AUC-ROC Curve

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr,tpr,thresholds=roc_curve(pred_abc,test_y)
roc_auc=auc(fpr,tpr)
plt.figure()
plt.plot(fpr,tpr,color='darkorange',lw=10,label='ROC Curve(area=%0.2f)'%roc_auc)
plt.plot([0,1],[0,1],color='navy',lw=10,linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Ada Boost Classifier')
plt.legend(loc="lower right")
plt.show()

# 3.Gradient Boosting Classifier:

In [ ]:
gbc = GradientBoostingClassifier(criterion='mse', n_estimators=100, learning_rate=0.1, loss='deviance')
gbc.fit(train_x,train_y)
gbc.score(train_x,train_y)
pred_gbc = gbc.predict(test_x)

print("Accuracy Score:",accuracy_score(test_y,pred_gbc)*100)
print("Classification report:",classification_report(test_y,pred_gbc)*100)
print("Confusion Matrix:",confusion_matrix(test_y,pred_gbc)*100)

gbc_score = cross_val_score(gbc,xtrain,ytrain,cv=9)
gbc_cc = gbc_score.mean() 
print('Cross Val Score:',gbc_cc*100)

AUC-ROC Curve:

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr,tpr,thresholds=roc_curve(pred_gbc,test_y)
roc_auc=auc(fpr,tpr)
plt.figure()
plt.plot(fpr,tpr,color='darkorange',lw=10,label='ROC Curve(area=%0.2f)'%roc_auc)
plt.plot([0,1],[0,1],color='navy',lw=10,linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Gradient Boosting Classifier')
plt.legend(loc="lower right")
plt.show()

# XGBOOST:

In [ ]:
#!pip install xgboost

In [ ]:
import xgboost

xgb = xgboost.XGBClassifier()
xgb.fit(train_x,train_y)
xgb.score(train_x,train_y)
pred_xgb = xgb.predict(test_x)

print("Accuracy Score:",accuracy_score(test_y,pred_xgb)*100)
print("Classification report:",classification_report(test_y,pred_xgb)*100)
print("Confusion Matrix:",confusion_matrix(test_y,pred_xgb)*100)

xgb_score = cross_val_score(xgb,xtrain,ytrain,cv=9)
xgb_cc = gbc_score.mean() 
print('Cross Val Score:',xgb_cc*100)

AUC-ROC Curve:

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr,tpr,thresholds=roc_curve(pred_xgb,test_y)
roc_auc=auc(fpr,tpr)
plt.figure()
plt.plot(fpr,tpr,color='darkorange',lw=10,label='ROC Curve(area=%0.2f)'%roc_auc)
plt.plot([0,1],[0,1],color='navy',lw=10,linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('XGBoost Classifier')
plt.legend(loc="lower right")
plt.show()

# The best model is Random Forest Classifier. Since the difference between the percentage score of cross validation,accuracy_score and ROC Curve is optimum.

# Model Saving:

In [ ]:
import pickle
filename='malign-comment.pkl'
pickle.dump(rfc,open(filename,'wb'))

# Coclusion:

In [ ]:
import numpy as np
a=np.array(test_y)
predicted=np.array(rfc.predict(test_x))
mal_con=pd.DataFrame({"original":a,"predicted":predicted},index=range(len(a)))
mal_con

From the above table, the model is predicted with 78 percent accuracy.


In [ ]:
#Converting test dataset into vector

test1=tfidf.fit_transform(test['comment_text'])
test1

In [ ]:
#predicting test dataset using chosen model

pred = rfc.predict(test1)
pred

In [ ]:
#creating dataset with predicted output

test['Predicted Output'] = pred
test

In [ ]:
#converting into csv file

test.to_csv('Test.csv', index = False)

In [ ]:
#downloading the csv file

test.to_csv(r'C:\Users\DELL\Downloads\Test.csv')